In [ ]:
from environment import Market, get_ts
from model import Q_Model
from agent import Agent
from simulator import Simulator
import pandas as pd

# sampler = Single_Signal_Generator(total_timesteps=180, period_range=(10, 40), amplitude_range=(5, 80), noise_amplitude_ratio=0.5)
# filename = "Generated Signals.npy"
# sampler.build_signals(filename, 1000)
# sampler.load(filename)
# env = Market(sampler=sampler, last_n_timesteps=40, buy_cost=3.3)

ticker=['AAPL', 'TSLA']
sample = get_ts(ticker)
sample.to_csv('data/test/test_data.csv')
# import os
# print(os.getcwd())
sample = pd.read_csv('data/test/test_data.csv', index_col=0)
print(sample.head())
# print(sample.shape)
# print(sample.dtypes)
env = Market(sample, 20, 3.3)
# env.reset()

dense_model = [
    {"type":"Reshape", "target_shape":(env.get_state().shape[0]*env.get_state().shape[1],)},
    {"type":"Dense", "units":128*env.get_state().shape[1]},
    {"type":"Dense", "units":32}
]
conv_model = [
    {"type":"Reshape", "target_shape":env.get_state().shape},
    {"type":"Conv1D", "filters":16, "kernel_size":3, "activation":"relu"},
    {"type":"Conv1D", "filters":16, "kernel_size":3, "activation":"relu"},
    {"type":"Flatten"},
    {"type":"Dense", "units":48, "activation":"relu"},
    {"type":"Dense", "units":24, "activation":"relu"}
]
gru_model = [
    {"type":"Reshape", "target_shape":env.get_state().shape},
    {"type":"GRU", "units":16, "return_sequences":True},
    {"type":"GRU", "units":16, "return_sequences":False},
    {"type":"Dense", "units":16, "activation":"relu"},
    {"type":"Dense", "units":16, "activation":"relu"}
]
lstm_model = [
    {"type":"Reshape", "target_shape":env.get_state().shape},
    {"type":"LSTM", "units":16, "return_sequences":True},
    {"type":"LSTM", "units":16, "return_sequences":False},
    {"type":"Dense", "units":16, "activation":"relu"},
    {"type":"Dense", "units":16, "activation":"relu"}
]

q_model = Q_Model("Dense", state_dim=env.get_state().shape, no_of_actions=env.no_of_actions, layers=dense_model, hyperparameters={"lr":0.0001})
agent = Agent(q_model, batch_size=8, discount_factor=0.995, epsilon=1)

no_of_episodes_train = 100
no_of_episodes_test = 1

sim = Simulator(env, agent)
sim.train(no_of_episodes_train, epsilon_decay=0.997)
agent.model.save()
sim.test(no_of_episodes_test)

Using TensorFlow backend.
  0%|          | 0/100 [00:00<?, ?it/s]

               AAPL   TSLA
date                      
2010-06-29  31.8384  23.89
2010-06-30  31.2617  23.83
2010-07-01  30.8827  21.96
2010-07-02  30.6913  19.20
2010-07-06  30.9013  16.11
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_1 (Reshape)          (None, 40)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               10496     
_________________________________________________________________
dense_2 (Dense)              (None, 32)                8224      
_________________________________________________________________
dense_3 (Dense)              (None, 10)                330       
Total params: 19,050
Trainable params: 19,050
Non-trainable params: 0
_________________________________________________________________
-----------------------------------------------------------

 48%|████▊     | 48/100 [55:35<1:16:57, 88.80s/it]